# 5) CPU Optimization: Matrix-Matrix Multiply

Last time:
- Intro to Vectorization

Today: 
1. Introduction to CPU Optimization  
  1.1 Matrix-Matrix Multiply
2. Loop ordering

## 1. Introduction to CPU Optimization

In this module we talk about optimizing matrix-matrix multiply on the CPU. Some of the material here is largely based on the [LAFF-On Programming for High Performance](https://www.cs.utexas.edu/~flame/laff/pfhp/LAFF-On-PfHP.html) of Robert van de Geijn,
Margaret Myers, Devangi Parikh, which has a pairing online MOOC: [LAFF-On Programming for High Performance (LAFF-On-PfHP)](https://www.cs.utexas.edu/users/flame/laff/pfhp/LAFF-On-PfHP.html).

I encourage you to watch the videos before class. When watching the videos, really you should be watching for the _big picture_ and we will revisit most of the material in lecture as well, mainly you will have a better learning experience if the in class lectures are the first time you've seen some of the material.

The MOOC is done using C (not Julia), so some of the syntax and language discussion is not relevant, but the overall ideas apply. 

Two big differences between the material presented in C Vs Julia:

1. Julia starts it's indexing with `1` whereas C uses `0`, so the first element of an array `A` in Julia is:

```julia
A[1]
```

whereas in C is:

```C
A[0]
```

2. The second difference is that in C, matrices are not
really first-class objects so they must be indexed as flat vectors. In Julia, even though behind the scenes everything is a flat vector, we can index into matrices directly.

Some additional Julia related material was gleaned from Sacha Verweij's repository [GemmDemo.jl](https://github.com/Sacha0/GemmDemo.jl) and from a former colleague, Maciej Waruszewski's repository [MyJuliaGEMM](https://github.com/mwarusz/MyJuliaGEMM).

Why do we care about CPU optimization (and not just MPI). Again. because of this:

![42 years of microprocessor data](https://www.karlrupp.net/wp-content/uploads/2018/02/42-years-processor-trend.png)

Source: [Microprocessor Trend Data](https://github.com/karlrupp/microprocessor-trend-data).

> Related videos and texts: 
> - [1.1.1 Launch, part 1](https://www.youtube.com/watch?v=knTLy1j-rco) (time 7:51), [text](https://www.cs.utexas.edu/~flame/laff/pfhp/week1-launch.html)
> - [GFLOPS](https://www.youtube.com/watch?v=cRyVMrNNRCk) (time: 5:31), [text](https://www.cs.utexas.edu/~flame/laff/pfhp/week1-launch.html)
> - [1.2.2  The leading dimension of a matrix](https://www.youtube.com/watch?v=PhjildK5oO8) 
(time: 4:02), [text](https://www.cs.utexas.edu/~flame/laff/pfhp/week1-the-leading-dimension.html).

### 1.1 Matrix-Matrix Multiply

We will focus on matrix-matrix multiplication:

```math
C = C + A * B
```
where $C$ is $m \times n$, $A$ is $m \times k$, and $B$ is $k \times n$.

The formula we learn for this in Linear Algebra is the following:

```math
γ_{ij} = γ_{ij} + \sum_{p=1}^{k} α_{ip} β_{pj}
```

where $γ_{ij}$, $α_{ip}$, and $β_{pj}$ are the elements of the matrices $C$, $A$, and $B$, respectively.

#### Notation
See [text 1.3.1 Notation](https://www.cs.utexas.edu/~flame/laff/pfhp/week1-notation.html)

 - Capital letters for matrices $A$, $B$, etc.
 - lower case letters for vectors $a$, $b$, etc.
 - lower case Greek letters for floating point scalars
   $α$ (corresponds to matrix $A$),
   $β$ (corresponds to matrix $B$),
   $γ$ (corresponds to matrix $C$),
 - lower case letters for integer scalars $n$, $m$, $k$ (for matrix sizes) and $i$, $j$, $p$ for element indices
 - Often useful to think of matrices as collections of column and/or row vectors: 
 
$$
 A = [a_1 a_2 \dots a_k] =
   \begin{bmatrix}
     \tilde{a}_{1}^{T}\\
     \tilde{a}_{2}^{T}\\
     \ddots\\
     \tilde{a}_{n}^{T}
   \end{bmatrix}
$$

(note that we think of all vectors as column vectors).

## 2. Loop ordering

See [text 1.2.4 Ordering the loops](https://www.cs.utexas.edu/~flame/laff/pfhp/week1-ordering-the-loops.html).

When the above formula is implemented as a triply nested loop:

```julia
  for i = 1:m
    for j = 1:n
      for p = 1:k
        C[i, j] += A[i, p] * B[p, j]
      end
    end
  end
```

The loop ordering does not impact the correctness of the algorithm. Each of the loop orderings can be interpreted slightly differently. 


### 2.1 Loop ordering: ijp

Matrix times row vector (update rows of $C$)

```math
  \tilde{c}_{i}^T = \tilde{c}_{i}^{T} + \tilde{a}_{i}^T B
```

with inner $\tilde{a}_{i}^T B$ computed using dot products (update $\gamma_{ij}$):

```math
   \tilde{a}_{i}^T B =
   \begin{bmatrix}
     \tilde{a}_{i}^T b_1 & \cdots & \tilde{a}_{i}^T b_m
   \end{bmatrix}
```

### 2.2 Loop ordering: ipj

Matrix times row vector (update rows of $C$)

```math
  \tilde{c}_{i}^T = \tilde{c}_{i}^{T} + \tilde{a}_{i}^T B
```

with inner $\tilde{a}_{i}^T B$ computed using `axpy`, scalar times vector plus vector (update $\tilde{c}_{i}$):

```math
   \tilde{a}_{i}^T B = \sum_{p=1}^{k} \alpha_{ip} \tilde{b}_p^T
```

### 2.3 Loop ordering: pij

Rank one update (repeatedly update all elements of $C$)

```math
  C = C + \sum_{p=1}^{k} a_{p} \tilde{b}_{p}^T
```

with outer product computed using axpy with vector $\tilde{b}_{p}^{T}$

```math
  a_{p} \tilde{b}_{p}^T =
  \begin{bmatrix}
    \alpha_{1p} \tilde{b}_{p}^T\\
    \vdots\\
    \alpha_{np} \tilde{b}_{p}^T
  \end{bmatrix}
```

### 2.4 Loop ordering: pji

Rank one update (repeatedly update all elements of $C$)

```math
  C = C + \sum_{p=1}^{k} a_{p} \tilde{b}_{p}^T
```

with outer product computed using axpy with vector $a_{p}$

```math
  a_{p} \tilde{b}_{p}^T =
  \begin{bmatrix}
    a_{p} \beta_{p1} & \cdots & a_{p} \beta_{pm}
  \end{bmatrix}
```

### 2.5 Loop ordering: jpi

Matrix times column vector (update columns of $C$)

```math
  c_j = c_{j} + A b_j
```

with inner $A b_j$ computed using `axpy`:

```math
  A b_j = \sum_{p=1}^{k} a_{p}\beta_{pj}
```

### 2.6 Loop ordering: jip

Matrix times column vector (update columns of $C$)

```math
  c_j = c_{j} + A b_j
```

with inner $A b_j$ computed using dot products

```math
  A b_j =
  \begin{bmatrix}
    \tilde{a}_{1}^{T} b_{j}\\
    \vdots\\
    \tilde{a}_{n}^{T} b_{j}\\
  \end{bmatrix}
```

